Data Wrangling Notebook for SY Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [251]:
import pandas as pd
from pandas import Series
import numpy as np
import re
import uuid 

Import Vertnet Mammal Data

In [252]:
sy_data = pd.read_csv("./../Original Data/futres_sy_original.csv")

Subset test data

In [253]:
test = sy_data[sy_data.columns[-4:]]

Rename Columns

In [254]:
test = test.rename(columns={'Measurements (NAME;NAME;NAME...) divided by ;':'measurementType',
                            'Values (mm;mm;mm...) divided by ;': 'measurementValue',
                            'PROTOCOL':'protocol',
                            'MEASUREMENTS':'measurements'})

Separate by ; and expand 

In [255]:
def matching_size(row):
    """function to match size of measurement type and value"""
    mtype = row["measurementType"].split(';')
    mval = row["measurementValue"].split(';')
    if len(mtype) == len(mval):
        line_split(row)
    else:
        print ("Error: Size Mismatch")

def line_split(row):
    mtype = row["measurementType"].split(';')
    mval = row["measurementValue"].split(';')
    temp_df = pd.DataFrame(row) 
    for i in range(len(mval)):
        temp_df["measurementType"] = mtype[i]
        temp_df["measurementValue"] = mval[i]
    print("herro")
    print(temp_df)
    print("meow")



test = test.apply(matching_size,axis=1)



#def calculate_area(row):
#    return row['height'] * row['width']
#rectangles_df.apply(calculate_area, axis=1)

#rectangles_df['area'] = rectangles_df.apply(calculate_area, axis=1)
#rectangles_df






#This method might be major issue for more than one index row, test that next
#I'd say this one is a possibility but it's not ideal 
#test_1 =  pd.concat([Series(row['protocol'], row['measurementValue'].split(';'))              
#                   for _, row in test.iterrows()]).reset_index()
#test_1 = test_1.rename(columns={'index':'measurementValue'})
#mval = test_1["measurementValue"]


#test_2 = pd.concat([Series(row['protocol'], row['measurementType'].split(';'))              
#                   for _, row in test.iterrows()]).reset_index()
#test_2 = test_2.rename(columns={'index':'measurementType',
#                                '0':'protocol'})

#test_2['measurementValue'] = mval
#test_2



#Focus on working measurementType instead

#test_1 = pd.concat([Series(row['var2'], row['var1'].split(','))              
#                    for _, row in a.iterrows()]).reset_index()


#test_1 = test.assign(var1=test['measurementType'].str.split(';')).explode('measurementType')

#test_1

#mtype = test['measurementType'].str.split(';').apply(Series, 1).stack()
#mtype.index = mtype.index.droplevel(-1) 
#mtype.name = 'measurementType' 

#mval = test['measurementValue'].str.split(';').apply(Series, 1).stack()
#mval.index = mval.index.droplevel(-1)
#mval.name = 'measurementValue'

#mval
#mtype

#del test['measurementType']
#test = test.join(mtype)



#Issue: caused same problem as the first one

#test =  pd.concat([Series(row['measurementType'], row['measurementValue'].split(';'))              
#                    for _, row in test.iterrows()]).reset_index()


#Issue: too much duplication, must be a better way 

#mtype = test['measurementType'].str.split(';').apply(Series, 1).stack()
#mval = test['measurementValue'].str.split(';').apply(Series, 1).stack()
#mtype.index = mtype.index.droplevel(-1) 
#mval.index = mval.index.droplevel(-1)
#mtype.name = 'measurementType' 
#mval.name = 'measurementValue'
#del test['measurementType']
#del test['measurementValue']
#test = test.join(mtype)
#test = test.join(mval)

herro
                                        0 measurementType measurementValue
measurements                           SI              Dd             17,2
protocol          L'Hereux y Borella 2011              Dd             17,2
measurementType                     Bd;Dd              Dd             17,2
measurementValue                29,5;17,2              Dd             17,2
meow
herro
                                       1 measurementType measurementValue
measurements                          SI              Dd                2
protocol          Von den Driesch (1976)              Dd                2
measurementType           GL;Bp;Bd;Dp;Dd              Dd                2
measurementValue            14;4,5;4;4;2              Dd                2
meow
herro
                                       2 measurementType measurementValue
measurements                          SI            HFcr              2,2
protocol          Von den Driesch (1976)            HFcr              2,2
measu

In [235]:
test

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

In [236]:
#test_new_val = test.join(test.pop('measurementValue').str.split(';',expand=True))
#test_new_val = test_new_val.set_index(['protocol','measurementType']).stack().reset_index(level=[0,1],name='measurementValue')

In [237]:
#test_new_type = test.join(test.pop('measurementType').str.split(';',expand=True))
#test_new_type = test_new_type.set_index(['protocol','measurementValue']).stack().reset_index(level=[0,1],name='measurementType')

In [238]:
test_new_val

NameError: name 'test_new_val' is not defined